# Failed Integration tests

In [4]:
# If you have problems with import modules, uncomment line below and run it *ONE* time
# Comment it out after the execution
# %cd ../

In [5]:
import requests
from data_generators.generators import RandomMentor, RandomMentee, RandomMenteeFeedback
from app.data import MongoDB


In [6]:
# API URL
url = "http://127.0.0.1:8000"

# Tag for marking test data
test_tag = 'test_INTEGRATION_test'

## Mentor Operations


In [7]:
# URL for the request

# Mentors
url_create_mentor = f'{url}/create/mentor'
url_read_mentor = f'{url}/read/mentor'
url_update_mentor = f'{url}/update/mentor/'


Setup for Mentors
Create a list of randomly generated mock mentor data and mark them as ‘test_INTEGRATION_test’ for post-testing cleanup.

In [8]:
# Set number of mentors to test
n_persons = 3
# Create list of test mentors
mentors = []

for i in range(n_persons):
    mentors.append(vars(RandomMentor()))
    mentors[i]['other_info'] = 'test_INTEGRATION_test'


### Try to create multiple test mentors with API
API Should return 422 code and *error*:
'msg': 'value is not a valid dict',
   'type': 'type_error.dict'

In [9]:
# API doesn't allow to create multiples entries
expected = {'msg': 'value is not a valid dict',
            'type': 'type_error.dict'}
expected_status = 422
ret = requests.post(url_create_mentor, json=mentors)

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected['msg'] == ret.json()['detail'][0]['msg'], f"Endpoint did not return expected value of {expected['msg']}"
assert expected['type'] == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected['type']}"


### Try to create duplicated record
API Should return 409 error:
`{'detail': 'Profile ID must be unique.'}`

In [10]:
# Try to create a DUPLICATE test mentor with API, we should receive 'result': True
expected = {'detail': 'Profile ID must be unique.'}
expected_status = 409


ret1 = requests.post(url_create_mentor, json=mentors[0])

assert ret1.status_code == 200

ret = requests.post(url_create_mentor, json=mentors[0])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert  expected == ret.json(), f"Endpoint did not return expected value of {expected}"

### Try to create record with field not in schema.
API should return 422 *error*  `value_error.extra`

In [11]:
# Create an extra test field
mentors[1]['test_FIELED_test'] = 'This a field for integration test'

expected = {'msg': 'extra fields not permitted',
            'type': 'value_error.extra'}
expected_status = 422

# Try to create mentor
ret = requests.post(url_create_mentor, json=mentors[1])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected['msg'] == ret.json()['detail'][0]['msg'], f"Endpoint did not return expected value of {expected['msg']}"
assert expected['type'] == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected['type']}"

### Try to create a record with wrong value (name > 255 chars)
API should return 422 error:
 `value_error.any_str.max_length`

In [12]:
# Create First name more than 255 legal chars
mentors[2]['first_name'] = 300 * 'a'

expected_status = 422
expected = { 'msg': 'ensure this value has at most 255 characters',
             'type': 'value_error.any_str.max_length'}
# Try to create a wrong mentor
ret = requests.post(url_create_mentor, json=mentors[2])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected['msg'] == ret.json()['detail'][0]['msg'], f"Endpoint did not return expected value of {expected['msg']}"
assert expected['type'] == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected['type']}"

### Try to read non-existent value.
Should return zero list


In [13]:
# Read from the API with non-existent name
expected = 0
expected_status = 200

ret = requests.post(url_read_mentor, json={'first_name': 'test_INEXISTINGNAME_test'})
assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert len(ret.json()['result']) == expected, f"Endpoint return list other than {expected}"


### Try to update with wrong value of 'current_position' more that 255 chars.


In [14]:
# Create a wrong value (more than 255 chars)
current_position_wrong = 300 * 'a'

# Update test Mentor current_company value
# Should return value_error
expected_status = 422
expected = 'value_error.any_str.max_length'

ret = requests.post(url_update_mentor  + mentors[0]['profile_id'], json={'current_company': current_position_wrong})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected['type']}"


## Mentee operations

In [15]:
# Mentees
url_create_mentee = f'{url}/create/mentee'
url_read_mentee = f'{url}/read/mentee'
url_update_mentee = f'{url}/update/mentee/'


### Generate a list of random *test* mentees

In [16]:
# Create list of test mentors
mentees = []

for i in range(n_persons):
    mentees.append(vars(RandomMentee()))

    # create a field to identify our test records later
    mentees[i]['other_info'] = 'test_INTEGRATION_test'


### Create multiple test mentees with API
API Should return *error*

In [17]:
# API doesn't allow to create multiples entries
expected_status = 422
expected = 'type_error.dict'

ret = requests.post(url_create_mentee, json=mentees)

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected}"

Failed test. Try to create duplicated recrord
API Should return:
`{'detail': 'Profile ID must be unique.'}`

In [18]:
# Try to create a DUPLICATE test mentor with API, we should receive 'result': True
expected_status = 409
expected = {'detail': 'Profile ID must be unique.'}

ret = requests.post(url_create_mentee, json=mentees[0])
ret = requests.post(url_create_mentee, json=mentees[0])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json(), f"Endpoint did not return expected value of {expected}"

### Try to create record with field not in schema. API should return error value_error.extra

In [19]:
# Create an extra test field
mentees[1]['test_FIELED_test'] = 'This a field for integration test'

expected_status = 422
expected = 'value_error.extra'

# Try to create mentor
ret = requests.post(url_create_mentee, json=mentees[1])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected}"

## Try to create a record with wrong value
API should return 422 error:
 `value_error.any_str.max_length`

In [20]:
# Create First name more than 255 legal chars
mentees[2]['first_name'] = 300 * 'a'

expected_status = 422
expected = 'value_error.any_str.max_length'

# Try to create a wrong mentor
ret = requests.post(url_create_mentee, json=mentees[2])

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json()['detail'][0]['type'], f"Endpoint did not return expected value of {expected}"


### Read non-existing value.
Should return zero list

In [21]:
expected_status = 200
expected = {'result': []}

# Read from the API with non-existing name
ret = requests.post(url_read_mentee, json={'first_name': 'test_INEXISTINGNAME_test'})

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert expected == ret.json(), f"Endpoint did not return expected value of {expected}"


## Feedback operations


In [22]:
# Feedback
url_create_feedback = f'{url}/create/feedback'
url_read_feedback = f'{url}/read/feedback'
url_update_feedback = f'{url}/update/feedback?ticket_id='
url_delete_feedback = f'{url}/delete/feedback?ticket_id='

### Generate random feedbacks

In [23]:
feedback = vars(RandomMenteeFeedback(mentees[i]['profile_id'], mentors[i]['profile_id']))
feedback['text'] = "test_INTEGRATION_test"

assert feedback['ticket_id'] != '', f"Something wrong with RandomMenteeFeedback() generator"


### Try to create a duplicate record


In [24]:
# There is a bug, that crashes server when creating a duplicate feedback

# Create ONE feedback
# expected_status = ...
# expected = ...
#
# ret = requests.post(url_create_feedback, json=feedback)
# ret = requests.post(url_create_feedback, json=feedback)
#
# assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
# assert ret.json() == expected, f"Endpoint did not return expected value of {expected}"

### Try to create multiple records


In [25]:
# There is a bug, that crashes server when creating a duplicate feedback

# Create ONE feedback
# expected_status = ...
# expected = ...
#
# ret = requests.post(url_create_feedback, json=feedbacks)
#
# assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
# assert ret.json() == expected, f"Endpoint did not return expected value of {expected}"

### Try to create feedback with not existent field

In [27]:
expected_status = 422
expected = 'value_error.extra'

feedback['test_NONEXISTENT_test'] = 'test_TEST_test'

ret = requests.post(url_create_feedback, json=feedback)

assert ret.status_code == expected_status, f"Return status code was not {expected_status}. Got status code {ret.status_code}"
assert ret.json()['detail'][0]['type'] == expected, f"Endpoint did not return expected value of {expected}"

# Cleaning test data


In [28]:
# Make a connection to the db
db = MongoDB()

# Mentors and Mentees clean up
for collection in ['Mentors', 'Mentees']:
    db.delete(collection, {'other_info': test_tag})

# Feedback clean up
db.delete('Feedback', {'text': test_tag})

### Check that we have successfully cleaned up the db.


In [29]:
# Mentors and Mentees
for collection in ['Mentors', 'Mentees']:
    ret = db.search(collection, test_tag)
    assert  ret == [], f"Clean up of {collection} failed, found db record: {ret}"

# Feedback
ret = db.read('Feedback', {'ticket_id': feedback['ticket_id']})
assert  ret == [], f"Clean up of Feedback failed, found db record: {ret}"
